In [232]:
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import yfinance as yf
from time import sleep

In [233]:
wiki = 'https://en.wikipedia.org/wiki/'

In [234]:
cac40_tickers = pd.read_html(wiki + 'CAC_40', flavor='html5lib')[4]['Ticker'].to_list()

In [235]:
def get_stock_price_history(tickers):
    data = []
    for ticker in tickers:
        data.append(yf.download(tickers=ticker,
                                period="5d",
                                interval="1m").reset_index())
    return data

In [236]:
cac40 = get_stock_price_history(cac40_tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [237]:
def new_engine(name):
    engine = create_engine(f'postgresql://pavelkurach@localhost:5432/{name}')
    if not database_exists(engine.url):
        create_database(engine.url)
    print(database_exists(engine.url))
    return engine

In [238]:
cac40_engine = new_engine('CAC40')

True


In [239]:
def to_sql(frames, tickers, engine):
    for frame, ticker in zip(frames, tickers):
        frame.to_sql(ticker, engine, if_exists='replace', index=False)
        with cac40_engine.connect() as con:
            con.execute(f'ALTER TABLE "{ticker}" ADD PRIMARY KEY ("{frame.columns[0]}");')

    print("Successfully created a database")

In [240]:
to_sql(cac40, cac40_tickers, cac40_engine)

Successfully created a database


In [241]:
Base = automap_base()

Base.prepare(autoload_with=cac40_engine)

In [242]:
cac40_classes = {ticker: cls for ticker, cls in zip(cac40_tickers, Base.classes)}

In [243]:
session = Session(cac40_engine)

In [244]:
query = session.query(cac40_classes['AI.PA'])

In [245]:
df = pd.read_sql(query.statement, query.session.bind, index_col='Datetime')
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-16 09:00:00,134.000000,134.080002,133.899994,134.039993,134.039993,0
2022-12-16 09:01:00,134.020004,134.199997,134.020004,134.100006,134.100006,1656
2022-12-16 09:02:00,134.059998,134.179993,134.059998,134.179993,134.179993,406
2022-12-16 09:03:00,134.300003,134.460007,134.300003,134.339996,134.339996,443
2022-12-16 09:04:00,134.259995,134.259995,134.059998,134.179993,134.179993,633
...,...,...,...,...,...,...
2022-12-22 16:51:00,133.500000,133.500000,133.479996,133.479996,133.479996,321
2022-12-22 16:52:00,133.479996,133.479996,133.460007,133.460007,133.460007,383
2022-12-22 16:53:00,133.479996,133.479996,133.460007,133.460007,133.460007,193


In [246]:
import matplotlib

df.Close.pct_change()

Datetime
2022-12-16 09:00:00         NaN
2022-12-16 09:01:00    0.000448
2022-12-16 09:02:00    0.000596
2022-12-16 09:03:00    0.001192
2022-12-16 09:04:00   -0.001191
                         ...   
2022-12-22 16:51:00   -0.000300
2022-12-22 16:52:00   -0.000150
2022-12-22 16:53:00    0.000000
2022-12-22 16:54:00   -0.000150
2022-12-22 16:55:00   -0.000450
Name: Close, Length: 2475, dtype: float64

In [247]:
import numpy as np

highs = df['High'] / df['Close'].shift() - 1
print(highs)

Datetime
2022-12-16 09:00:00         NaN
2022-12-16 09:01:00    0.001194
2022-12-16 09:02:00    0.000596
2022-12-16 09:03:00    0.002087
2022-12-16 09:04:00   -0.000596
                         ...   
2022-12-22 16:51:00   -0.000150
2022-12-22 16:52:00    0.000000
2022-12-22 16:53:00    0.000150
2022-12-22 16:54:00    0.000000
2022-12-22 16:55:00   -0.000150
Length: 2475, dtype: float64


In [248]:
lows = df['Low'] / df['Close'].shift() - 1
np.sum(lows < -3 * lows.std())

43

In [249]:
import datetime as dt

print(dt.datetime.now())
print(df.index[0] < dt.datetime.now())
start = df.index[-1]
start

2022-12-22 17:11:16.627797
True


Timestamp('2022-12-22 16:55:00')

In [250]:
yf.download(tickers='AI.PA',
            period="1d",
            interval="1m").reset_index()

[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-12-22 09:00:00,135.339996,135.559998,135.300003,135.440002,135.440002,0
1,2022-12-22 09:01:00,135.539993,135.559998,135.539993,135.559998,135.559998,50
2,2022-12-22 09:02:00,135.479996,135.660004,135.479996,135.660004,135.660004,329
3,2022-12-22 09:03:00,135.580002,135.639999,135.559998,135.639999,135.639999,329
4,2022-12-22 09:04:00,135.619995,135.619995,135.619995,135.619995,135.619995,21
...,...,...,...,...,...,...,...
460,2022-12-22 16:52:00,133.479996,133.479996,133.460007,133.460007,133.460007,383
461,2022-12-22 16:53:00,133.479996,133.479996,133.460007,133.460007,133.460007,193
462,2022-12-22 16:54:00,133.460007,133.460007,133.419998,133.440002,133.440002,143
463,2022-12-22 16:55:00,133.419998,133.419998,133.380005,133.380005,133.380005,579


In [251]:
sigma = {}
for ticker in cac40_tickers:
    query = session.query(cac40_classes[ticker])
    temp_df = pd.read_sql(query.statement, query.session.bind, index_col='Datetime')
    sigma[ticker] = df.Close.pct_change().std()

print(sigma)

{'AI.PA': 0.0004494217199336799, 'AIR.PA': 0.0004494217199336799, 'ALO.PA': 0.0004494217199336799, 'MT.AS': 0.0004494217199336799, 'CS.PA': 0.0004494217199336799, 'BNP.PA': 0.0004494217199336799, 'EN.PA': 0.0004494217199336799, 'CAP.PA': 0.0004494217199336799, 'CA.PA': 0.0004494217199336799, 'ACA.PA': 0.0004494217199336799, 'BN.PA': 0.0004494217199336799, 'DSY.PA': 0.0004494217199336799, 'ENGI.PA': 0.0004494217199336799, 'EL.PA': 0.0004494217199336799, 'ERF.PA': 0.0004494217199336799, 'RMS.PA': 0.0004494217199336799, 'KER.PA': 0.0004494217199336799, 'OR.PA': 0.0004494217199336799, 'LR.PA': 0.0004494217199336799, 'MC.PA': 0.0004494217199336799, 'ML.PA': 0.0004494217199336799, 'ORA.PA': 0.0004494217199336799, 'RI.PA': 0.0004494217199336799, 'PUB.PA': 0.0004494217199336799, 'RNO.PA': 0.0004494217199336799, 'SAF.PA': 0.0004494217199336799, 'SGO.PA': 0.0004494217199336799, 'SAN.PA': 0.0004494217199336799, 'SU.PA': 0.0004494217199336799, 'GLE.PA': 0.0004494217199336799, 'STLA.PA': 0.00044942

In [253]:
update = {ticker: session.query(cac40_classes[ticker]).order_by(
    cac40_classes[ticker].Datetime.desc()).first().Datetime for ticker in cac40_tickers}

while True:
    for ticker in cac40_tickers:
        new_data = yf.download(tickers=ticker,
                               period="1d",
                               interval="1m", progress=False).reset_index()
        new_data = new_data[new_data['Datetime'] > update[ticker]]
        for _, tick in new_data.iterrows():
            last_close = session.query(cac40_classes[ticker]).order_by(
                cac40_classes[ticker].Datetime.desc()).first().Close
            change = tick["Close"] / last_close - 1
            if change > 2*sigma[ticker]:
                print(f'{ticker} stock price moved by {change * 100:.2f}% '
                      f'from {update[ticker].hour}:{update[ticker].minute} '
                      f'to {tick["Datetime"].hour}:{tick["Datetime"].minute}.')
                sleep(5)
            t = cac40_classes[ticker](**tick.to_dict())
            session.add(t)
            session.commit()
            update[ticker] = tick['Datetime']
    sleep(30)

DSY.PA stock price moved by 0.13% from 16:55 to 16:56.
TTE.PA stock price moved by 0.15% from 16:55 to 16:56.


KeyboardInterrupt: 

In [197]:
print(update.hour)

16


In [226]:
query = session.query(cac40_classes['AIR.PA']).order_by(
    cac40_classes['AIR.PA'].Datetime.desc()).limit(10)
pd.read_sql(query.statement, query.session.bind, index_col='Datetime')

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-22 16:35:00,111.760002,111.820000,111.660004,111.760002,111.760002,2865
2022-12-22 16:30:00,111.820000,111.839996,111.699997,111.739998,111.739998,5507
2022-12-22 16:25:00,111.760002,111.879997,111.760002,111.820000,111.820000,3450
2022-12-22 16:20:00,112.080002,112.080002,111.760002,111.760002,111.760002,5950
2022-12-22 16:15:00,22.700001,22.730000,22.690001,22.730000,22.730000,2070
2022-12-22 16:10:00,22.719999,22.719999,22.690001,22.709999,22.709999,11750
2022-12-22 16:05:00,22.740000,22.750000,22.700001,22.719999,22.719999,9577
2022-12-22 16:00:00,22.850000,22.850000,22.750000,22.750000,22.750000,45947
2022-12-22 15:55:00,22.830000,22.860001,22.830000,22.860001,22.860001,13865


In [ ]:
dt = session.query(cac40_classes['AI.PA']).order_by(cac40_classes['AI.PA'].Datetime.desc()).first().Datetime
dt

In [51]:
session.query(cac40_classes['AI.PA']).order_by(cac40_classes['AI.PA'].Datetime.desc()).first().Close

133.82000732421875